In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_BRONZE = PROJECT_ROOT / "data" / "bronze"
DATA_SILVER = PROJECT_ROOT / "data" / "silver"
DATA_SILVER.mkdir(parents=True, exist_ok=True)

train_path = DATA_BRONZE / "train.csv"
store_path = DATA_BRONZE / "store.csv"

print("Root:", PROJECT_ROOT)
print("Train:", train_path.exists())
print("Store:", store_path.exists())

Root: /Users/edonaire/Documents/governance-first-sales-prediction
Train: True
Store: True


In [2]:
train = pd.read_csv(train_path, low_memory=False)
store = pd.read_csv(store_path, low_memory=False)

print(train.shape)
print(store.shape)

train.head()

(1017209, 9)
(1115, 10)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [3]:
train["Date"] = pd.to_datetime(train["Date"], errors="coerce")

print("Invalid dates:", train["Date"].isna().sum())

Invalid dates: 0


In [4]:
df = train.merge(store, on="Store", how="left")

print(df.shape)
df.head()

(1017209, 18)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
# Flags binárias
bin_cols = ["Open", "Promo", "SchoolHoliday", "Promo2"]

for c in bin_cols:
    if c in df.columns:
        df[c] = df[c].fillna(0).astype(int)


# StateHoliday
if "StateHoliday" in df.columns:
    df["StateHoliday"] = (
        df["StateHoliday"]
        .astype(str)
        .replace({"0": "0", "nan": "0"})
        .fillna("0")
    )


# Competition distance
if "CompetitionDistance" in df.columns:
    df["CompetitionDistance"] = df["CompetitionDistance"].fillna(
        df["CompetitionDistance"].median()
    )


# Datas de competição/promo
date_cols = [
    "CompetitionOpenSinceMonth",
    "CompetitionOpenSinceYear",
    "Promo2SinceWeek",
    "Promo2SinceYear"
]

for c in date_cols:
    if c in df.columns:
        df[c] = df[c].fillna(0).astype(int)

In [6]:
df.isna().mean().sort_values(ascending=False).head(15)

PromoInterval                0.499436
DayOfWeek                    0.000000
Promo2SinceYear              0.000000
Promo2SinceWeek              0.000000
Promo2                       0.000000
CompetitionOpenSinceYear     0.000000
CompetitionOpenSinceMonth    0.000000
CompetitionDistance          0.000000
Assortment                   0.000000
Store                        0.000000
SchoolHoliday                0.000000
StateHoliday                 0.000000
Promo                        0.000000
Open                         0.000000
Customers                    0.000000
dtype: float64

In [7]:
silver_path = DATA_SILVER / "train_clean.parquet"

df.to_parquet(silver_path, index=False)

print("Saved:", silver_path)
print("Size (MB):", round(silver_path.stat().st_size / 1024**2, 2))

Saved: /Users/edonaire/Documents/governance-first-sales-prediction/data/silver/train_clean.parquet
Size (MB): 4.72


In [8]:
df.shape

(1017209, 18)

In [9]:
df.dtypes

Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth             int64
CompetitionOpenSinceYear              int64
Promo2                                int64
Promo2SinceWeek                       int64
Promo2SinceYear                       int64
PromoInterval                        object
dtype: object